## Observação:
Os arquvivoos estão sendo salvos em formato Delta, devido a importancia da garantia da eficiência e confiabilidade dos dados. Dessa forma, temos consultas melhoradas e alterações seguras.

In [0]:


# ====================================================
# 0. Importando bibliotecas necessárias
# ====================================================

# Para fazer chamadas HTTP às APIs
import requests
# Para manipular dados JSON (serialização/desserialização)
import json
# Para capturar a data/hora atual no momento da ingestão
import datetime
# Criar sessão Spark
from pyspark.sql import SparkSession
# Funções úteis do Spark SQL para manipulação de colunas e listas
from pyspark.sql.functions import col, explode, row_number, regexp_extract
# Para trabalhar com janelas (window functions, ex: deduplicação)
from pyspark.sql.window import Window

# ====================================================
# 1. Criando sessão Spark
# ====================================================
spark = SparkSession.builder.appName("PokemonCollector").getOrCreate()

# ====================================================
# 2. Função para coletar lista de Pokémon (endpoint principal da API)
# ====================================================
def get_pokemon_list(limit=100, max_records=300):
    """
    Faz chamadas paginadas à PokéAPI e retorna a lista de Pokémon.
    - limit = quantidade de registros por página (default = 100)
    - max_records = máximo de registros a coletar
    """
    url = "https://pokeapi.co/api/v2/pokemon/"  # endpoint base
    offset = 0  # deslocamento inicial para paginação
    all_data = []  # lista que armazenará todos os dados coletados

    # loop até atingir o máximo definido
    while offset < max_records:
        print(f"🔎 Coletando lista offset={offset}")
        # chamada GET com paginação (limit + offset)
        resp = requests.get(url, params={"limit": limit, "offset": offset})
        
        # se a API retornar erro, interrompe
        if resp.status_code != 200:
            print(f"❌ Erro: {resp.status_code}")
            break
        
        # converte resposta em JSON
        data = resp.json()
        # adiciona coluna de data/hora da ingestão
        data["ingestion_date"] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        # guarda os dados na lista final
        all_data.append(data)
        
        # se não existir link "next" na resposta, não há mais páginas
        if not data.get("next"):  
            break
        
        # incrementa offset para próxima chamada
        offset += limit

    return all_data

# ====================================================
# 3. Função para coletar detalhes de cada Pokémon
# ====================================================
def get_pokemon_details(url_list):
    """
    Recebe uma lista de URLs e coleta os detalhes de cada Pokémon individualmente.
    Retorna uma lista de dicionários.
    """
    details = []
    for url in url_list:  # percorre todas as URLs de Pokémon
        print(f"📥 Coletando detalhes de {url}")
        resp = requests.get(url)
        if resp.status_code == 200:  # sucesso na requisição
            data = resp.json()
            # adiciona timestamp de ingestão
            data["ingestion_date"] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            details.append(data)
        else:
            print(f"❌ Erro ao buscar {url}: {resp.status_code}")
    return details

# ====================================================
# 4. Função para serializar campos aninhados em JSON
# ====================================================
def serialize_nested_fields(data, fields):
    """
    Transforma listas/dicionários (nested fields) em strings JSON,
    para que o Spark consiga salvar corretamente.
    """
    for d in data:  # percorre cada Pokémon
        for f in fields:  # percorre cada campo aninhado
            if f in d:
                d[f] = json.dumps(d[f])  # transforma em string JSON
    return data

# ====================================================
# 5. Coleta lista de Pokémon (camada Bronze)
# ====================================================
# chama a função de ingestão
data_list = get_pokemon_list(limit=100, max_records=150)
# cria um DataFrame Spark a partir da lista de dicionários
df_bronze = spark.createDataFrame(data_list)
# salva a tabela no Delta Lake (camada bronze)
df_bronze.write.format("delta").mode("overwrite").saveAsTable("workspace.bronze.pokemon")
print("✅ Bronze salvo!")

# ====================================================
# 6. Transforma lista em Silver (explode + deduplicação)
# ====================================================
df_silver = (
    df_bronze
    # explode → quebra lista de "results" em várias linhas
    .withColumn("result", explode(col("results")))
    # seleciona apenas colunas de interesse
    .select(
        col("ingestion_date"),
        col("result.name").alias("name"),  # nome do Pokémon
        col("result.url").alias("url")     # link para detalhes
    )
)

# Deduplicação: mantém apenas o registro mais recente por Pokémon
